## Reducer: (Put your name here)

## Reviewer: (Put your name here)

# jupyter notebook crash course

1. Click on a code cell (has grey background) then, 
2. Press Shift-Enter (at the same time) to run a code cell. 

That will add the controls (buttons, etc) you use to do the reduction one-by-one; then use them for reduction.

In [ ]:
from __future__ import division, print_function

import reducer.gui
import reducer.astro_gui as astro_gui
from reducer.image_browser import ImageBrowser

from ccdproc import ImageFileCollection

from reducer import __version__
print(__version__)

In [ ]:
astro_gui.DEFAULT_MEMORY_LIMIT = 3e8

# This demo uses data pre-packaged with reducer

That low-resolution dataset will expand to about 300MB when uncompressed.

To use your own data, replace `reducer.notebook_dir.get_data_path()` with the path (folder location) of your data. That will not work on binder unless you upload your data.

In [ ]:
!wget http://physics.mnstate.edu/craig/ast366_f18.tgz
!tar xvf ast366_f18.tgz
!rm ast366_f18.tgz

In [ ]:
# To use the sample data set:
data_dir = 'ast366_f18'

# Or, uncomment line below and modify as needed
# data_dir = 'path/to/your/data'


## The cell below controls where the reduced images are saved

A dot, `.`, means the directory in which this notebook is stored, but you can set it to anything you want *except* the directory where the raw data is.

In [ ]:
destination_dir = '.'

# Type any comments about this dataset here

These are images of the Landolt field SA112.

# Load your data set

In [ ]:
images = ImageFileCollection(location=data_dir, keywords='*')

# Image Summary

## Includes browser and image/metadata viewer

This is not, strictly speaking, part of reduction, but is a handy way to take a quick look at your files. You can change the properties by which the images are organized by changing the list of `keys` below. Each key should be a keyword in the header of the FITS files.

In [ ]:
fits_browser = ImageBrowser(images, keys=['imagetyp', 'exposure'])
fits_browser.display()

# Make a master bias image

## Reduce the bias images

In [ ]:
bias_reduction = astro_gui.Reduction(description='Reduce bias frames',
                                     toggle_type='button',
                                     allow_bias=False,
                                     allow_dark=False,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     apply_to={'imagetyp': 'bias'},
                                     destination=destination_dir)
bias_reduction.display()

## Combine bias images to make master bias

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
bias_settings = astro_gui.Combiner(description="Master Bias Settings",
                                   toggle_type='button',
                                   file_name_base='master_bias',
                                   image_source=reduced_collection,
                                   apply_to={'imagetyp': 'bias'},
                                   destination=destination_dir)
bias_settings.display()

# Make a master dark

## Reduce dark images

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
dark_reduction = astro_gui.Reduction(description='Reduce dark frames',
                                     toggle_type='button',
                                     allow_bias=True,
                                     master_source=reduced_collection,
                                     allow_dark=False,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     destination=destination_dir,
                                     apply_to={'imagetyp': 'dark'})

dark_reduction.display()

## Combine reduced darks to make master(s)

Note the ``Group by`` option in the controls that appear after you execute the cell below. `reducer` will make a master for each value of the FITS keyword listed in ``Group by``. By default this keyword is named ``exposure`` for darks, so if you have darks with exposure times of 10 sec, 15 sec and 120 sec you will get three master darks, one for each exposure time.

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

dark = astro_gui.Combiner(description="Make Master Dark",
                          toggle_type='button',
                          file_name_base='master_dark',
                          group_by='exposure',
                          image_source=reduced_collection,
                          apply_to={'imagetyp': 'dark'},
                          destination=destination_dir)
dark.display()

# Make master flats

## Reduce flat images

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
flat_reduction = astro_gui.Reduction(description='Reduce flat frames',
                                     toggle_type='button',
                                     allow_bias=True,
                                     master_source=reduced_collection,
                                     allow_dark=True,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     destination=destination_dir,
                                     apply_to={'imagetyp': 'flat'})

flat_reduction.display()

## Build masters by combining

Again, note the presence of ``Group by``. If you typically use twilight flats you will almost certainly want to group by filter, not by filter and exposure.

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

flat = astro_gui.Combiner(description="Make Master Flat",
                          toggle_type='button',
                          file_name_base='master_flat',
                          group_by='filter',
                          image_source=reduced_collection,
                          apply_to={'imagetyp': 'flat'},
                          destination=destination_dir)
flat.display()

# Reduce the science images

There is some autmatic matching going on here:

+ If darks are subtracted a dark of the same edxposure time will be used, if available. If not, and dark scaling is enabled, the dark with the closest exposure time will be scaled to match the science image.
+ If the dark you want to scale appears not to be bias-subtracted an error will be raised.
+ Flats are matched by filter.

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
light_reduction = astro_gui.Reduction(description='Reduce light frames',
                                      toggle_type='button',
                                      allow_cosmic_ray=True,
                                      master_source=reduced_collection,
                                      input_image_collection=images,
                                      destination=destination_dir,
                                      apply_to={'imagetyp': 'light'})

light_reduction.display()

## Wonder what the reduced images look like? Make another image browser...

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

In [ ]:
fits_browser = ImageBrowser(reduced_collection, keys=['imagetyp', 'exposure'])
fits_browser.display()